In [ ]:
from numpy import linalg as LA
import numpy as np
import pandas as pd
import math
import networkx as nx
import community
import community.community_louvain as community_louvain

In [ ]:
def compute_C_minus_C0(lambdas,v,lambda_plus):
    N=len(lambdas)
    C_clean=np.zeros((N, N))
    
    v_m=np.matrix(v)
    
    # _s stands for _structure below: WHY N-1?!?!? isn't range enough
    for i in range(N-1):
        if lambdas[i]>lambda_plus:
            C_clean=C_clean+lambdas[i] * np.dot(v_m[:,i],v_m[:,i].T)  
    return C_clean    
    

In [79]:
def LouvainCorrelationClustering(R):   # R is a matrix of return
    
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2

    C=R.corr()
    lambdas, v = LA.eigh(C)

    order = np.argsort(lambdas)
    lambdas,v = lambdas[order],v[:,order]
      
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    mygraph= nx.from_numpy_matrix(np.abs(C_s))
    partition = community.community_louvain.best_partition(mygraph)
    DF=pd.DataFrame.from_dict(partition,orient="index")
    DF['StockName'] = R.columns
    DF = DF.reset_index()
    DF = DF.drop(columns = 'index')
    DF = DF.rename({0: 'Cluster'},axis = 'columns')
    DF = DF.set_index('StockName')
    return(DF)

In [80]:
clust_louvain=LouvainCorrelationClustering(DF_cut)
print(clust_louvain)
print(clust_louvain.value_counts()) # 6 clusters

           Cluster
StockName         
^GSPC            0
GE               0
IBM              0
DIS              0
BA               2
...            ...
SUI              2
MAA              2
RYN              2
RKT              7
EXP              1

[234 rows x 1 columns]
Cluster
3          43
2          39
7          34
4          32
1          29
0          20
6          20
5          17
dtype: int64


In [81]:
import numpy as np

DF=pd.read_parquet("us_equities_logreturns.parquet")

DF_cut=DF.iloc[-5000:]
display(DF)

sel=DF_cut.isnull().sum(axis=0)>0
DF_cut=DF_cut.drop(columns=DF_cut.columns[sel])
display(DF_cut)

,^GSPC,GE,IBM,DIS,BA,CAT,AA,HPQ,DD,KO,...,NSM,CLP,CTX,CTR,DYN,AIB,KIM,SFN,TCO,S
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.011340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.004737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.002949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.005872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16150,0.000305,-0.005420,-0.002896,0.004063,-0.009916,-0.000207,-0.001642,-0.010756,0.006343,-0.000520,...,0.042972,NaN,0.001974,-0.001879,0.011547,0.003892,0.002334,0.011191,0.009188,0.000000
16151,-0.011770,-0.016439,-0.012537,-0.017978,-0.020624,-0.012576,-0.025803,-0.023249,-0.010898,-0.013082,...,-0.011131,NaN,0.007076,-0.006604,-0.008647,-0.001296,0.001863,0.001534,-0.000436,-0.018391
16152,-0.002826,-0.009118,-0.009232,0.001750,0.009959,-0.002304,-0.001688,0.005519,0.000913,0.005253,...,0.047769,NaN,0.007026,0.003307,0.008237,0.003020,0.001395,0.005350,-0.015368,0.018391
16153,0.009568,0.012663,0.019565,0.016351,0.018590,0.002931,0.005055,0.014005,0.009232,0.002616,...,-0.017531,NaN,0.003882,0.007987,0.002458,0.025942,0.000465,0.009860,0.000442,-0.001140


,^GSPC,GE,IBM,DIS,BA,CAT,AA,DD,XOM,PG,...,KOF,MOV,IT,CBL,SU,SUI,MAA,RYN,RKT,EXP
11155,0.008308,0.002176,0.017529,0.012693,0.005808,0.010850,-0.003367,0.006540,0.011809,0.020834,...,0.065687,0.000000,0.044872,0.012423,-0.020409,0.000000,0.010598,0.012461,-0.018245,0.009449
11156,-0.010186,-0.019759,-0.013118,0.014312,-0.016873,0.003591,-0.001688,0.001954,-0.011809,-0.025523,...,-0.005312,0.028676,-0.028105,-0.008265,0.000000,0.000000,0.008996,-0.018751,-0.018584,0.000000
11157,0.005106,0.006630,0.000000,-0.005343,0.000000,-0.005992,-0.003384,0.008422,0.009672,0.020931,...,0.017162,-0.017826,0.007101,-0.004158,0.000000,0.011429,-0.004488,-0.003160,0.054746,0.009360
11158,0.000878,0.015301,0.004392,0.000000,-0.005907,0.013135,0.018472,0.012821,0.000000,0.000000,...,0.016873,-0.010850,0.000000,0.004158,0.000000,0.000000,0.010440,0.012579,0.016511,0.000000
11159,0.000788,-0.006529,0.017377,0.002675,0.000000,0.019965,-0.003333,0.010140,-0.007515,0.015982,...,-0.003868,0.018019,-0.007101,-0.012526,0.000000,-0.005698,-0.005952,-0.012579,0.034338,-0.028348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16150,0.000305,-0.005420,-0.002896,0.004063,-0.009916,-0.000207,-0.001642,0.006343,0.002125,0.003666,...,-0.023756,-0.002135,0.005901,0.015648,0.007896,0.004496,0.005812,-0.006601,-0.013024,-0.004196
16151,-0.011770,-0.016439,-0.012537,-0.017978,-0.020624,-0.012576,-0.025803,-0.010898,-0.006069,-0.000631,...,-0.012413,0.002372,-0.017392,-0.001727,-0.008506,-0.002471,-0.000595,-0.001546,-0.015742,-0.037990
16152,-0.002826,-0.009118,-0.009232,0.001750,0.009959,-0.002304,-0.001688,0.000913,-0.001817,-0.002655,...,0.010514,0.008493,0.004168,-0.013921,-0.003055,0.002471,0.002228,0.001105,-0.003880,0.015695
16153,0.009568,0.012663,0.019565,0.016351,0.018590,0.002931,0.005055,0.009232,0.009053,0.010830,...,0.010300,0.003985,0.005117,-0.008211,0.000000,0.000897,0.003406,-0.003761,0.000190,0.014880


## Louvain clustering

In [103]:
clust_louvain=LouvainCorrelationClustering(DF_cut)
#print(clust_louvain)
#print(clust_louvain.value_counts()) # 6 clusters

def compute_clusters_returns(df,clusters):
    means = df.mean(axis = 0).transpose().to_frame()
    means = pd.merge( means, clusters,left_index = True, right_on='StockName')
    means = means.groupby('Cluster').mean()
    return means
    
compute_clusters_returns(DF_cut,clust_louvain)

                0
Cluster          
0        0.000300
1        0.000392
2        0.000284
3        0.000410
4        0.000477
5        0.000328
6        0.000274
7        0.000418
<class 'pandas.core.frame.DataFrame'>


## Maximum likelihood clustering



There is no code for this method in Python. The following shows how to call R from Python.

In [4]:
# first install R https://cran.rstudio.com/
# # run this cell the first time you use rpy2 only.

from rpy2.robjects.packages import importr
utils = importr('utils')
utils.install_packages("data.table")
 

--- Per piacere, seleziona un mirror CRAN per la sessione ---


From cffi callback <function _consolewrite_ex at 0x00000205F6EA3948>:
Traceback (most recent call last):
  File "C:\Users\mauro\Anaconda3\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 132, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "C:\Users\mauro\Anaconda3\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 133, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 78: invalid continuation byte
R[write to console]: trying URL 'https://cran.mirror.garr.it/CRAN/bin/windows/contrib/4.1/data.table_1.14.2.zip'

R[write to console]: Content type 'application/zip'
R[write to console]:  length 2600426 bytes (2.5 MB)

R[write to console]: downloaded 2.5 MB




package 'data.table' successfully unpacked and MD5 sums checked


R[write to console]: Warning:
R[write to console]:  cannot remove prior installation of package 'data.table'

R[write to console]: Warning:
R[write to console]:  restored 'data.table'




The downloaded binary packages are in
	C:\Users\mauro\AppData\Local\Temp\RtmpW6bR6R\downloaded_packages


<rpy2.rinterface_lib.sexp.NULLType object at 0x00000205EF000588> [RTYPES.NILSXP]

In [9]:
import rpy2.robjects as robjects
r = robjects.r

#importr("data.table")
newfunc = r.source("""libClusteringGiadaMarsiliFast.R""")
aggregateClusters=robjects.globalenv["aggregateClusters"]   # now aggregateClusters is a Python function


R[write to console]: Error in file(filename, "r", encoding = encoding) : 
  cannot open the connection

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In file(filename, "r", encoding = encoding) :
R[write to console]: 
 
R[write to console]:  cannot open file 'libClusteringGiadaMarsiliFast.R': No such file or directory



RRuntimeError: Error in file(filename, "r", encoding = encoding) : 
  cannot open the connection


In [28]:
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

C=DF_cut.corr()


In [31]:
clust_ML=aggregateClusters(C.values)

NameError: name 'aggregateClusters' is not defined

In [ ]:
clust_ML[3]   # this is a dictionary (List in the R language) which contains the membership of each cluster, one cluster at a time

In [ ]:
np.array(clust_ML[3][0]) # first cluster

In [ ]:
pd.DataFrame(np.array(clust_ML.rx["s_i"][0]).flatten()).value_counts() # 49 clusters, of highly heterogeneous size